In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Aim is to predict the marks of students of the test data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Test data.xlsx to Test data.xlsx
Saving Training data.xlsx to Training data (1).xlsx


In [ ]:
# Use the file namd 'training data' to train the model

data = pd.read_excel('Training data.xlsx')
x_train = np.array(data.iloc[:,0:8])
y_train = np.array(data.iloc[:,8]).reshape(-1,1)

# Try plotting y_train with different features
# To get an idea whether to add some features or not
# Add some features if required in x_train

# Also do label encoding for features not represented in numbers
# refer the link if not know : https://youtu.be/589nCGeWG1w?si=t2Wa7LgbUOO4RooM

def feature_changing(x_train):
  # ---------
    # Converting 'internet' to numeric
    x_train[:, 0] = np.where(x_train[:, 0] == 'yes', 1, 0)

    # Converting 'sex' to numeric
    x_train[:, 1] = np.where(x_train[:, 1] == 'M', 1, 0)

    # Converting remaining columns to float
    x_train = x_train.astype(float)

    # Adding polynomial features
    x_train = np.column_stack((x_train, x_train[:, 2:8] ** 2, x_train[:, 2:8] ** 3))
  # ---------
    return x_train

x_train = feature_changing(x_train)

In [ ]:
def z_score(x_train):

  # ---------
    x_mean = np.mean(x_train, axis=0)
    x_std = np.std(x_train, axis=0)
    x_std[x_std == 0] = 1  # Avoid division by zero
    x_scaled = (x_train - x_mean) / x_std
  # ---------

    return x_scaled,x_std,x_mean

In [ ]:
def cost(x_train,y_train,w,b):

  # ---------
    m = x_train.shape[0]
    predictions = np.dot(x_train, w) + b
    loss = np.mean((predictions - y_train) ** 2) / 2
  # ---------

    return loss

In [ ]:
def gradient_descent(x_train, y_train, w, b, learning_rate=0.01, iterations=1000):

  # ---------
    m = x_train.shape[0]
    for _ in range(iterations):
        predictions = np.dot(x_train, w) + b
        dw = np.dot(x_train.T, (predictions - y_train)) / m
        db = np.sum(predictions - y_train) / m
        w -= learning_rate * dw
        b -= learning_rate * db
  # ---------

    return w,b

In [ ]:
x_train = x_train.astype(np.float64)
x_train,x_std,x_mean = z_score(x_train)

np.random.seed(2147483647)
w = np.random.randn(x_train.shape[1],1)
b = np.random.randn(1)

old_cost = 0

while abs(old_cost - cost(x_train,y_train,w,b))>0.00001:
  old_cost = cost(x_train,y_train,w,b)
  w,b = gradient_descent(x_train,y_train,w,b)

x_predict = pd.read_excel('Test data.xlsx').iloc[:,:8].to_numpy()
x_predict = feature_changing(x_predict)
x_predict = (x_predict - x_mean)/x_std
ans = pd.read_excel('Test data.xlsx').iloc[:,8].to_numpy()

y_predict = np.dot(x_predict,w) + b

accuracy = 0
for dim in range(len(ans)):
  if abs(y_predict[dim]-ans[dim])<0.5: # do not change the tolerance as you'll be checked on +- 0.5 error only
    accuracy += 1
accuracy = round(accuracy*100/200.0,2)
ok = 'Congratulations' if accuracy>95 else 'Optimization required'
print(f"{ok}, your accuracy is {accuracy}%")

Congratulations, your accuracy is 100.0%
